# Lecture 1: The Fourier Expansion and Orthogonality of Characters

**CS294-92: Analysis of Boolean Functions (Spring 2025)**  
**Instructor: Avishay Tal**  
**Notes by: Joyce Lu**

This notebook follows along with Lecture 1, demonstrating concepts using the `boofun` library.

---

## Course Overview

This course covers the **analysis of Boolean functions**, with applications to:

- **Property Testing** [BLR90]: Testing if functions have certain properties
- **Social Choice**: Arrow's Theorem, KKL Theorem, Friedgut's Junta Theorem
- **Learning Theory** [LMN93]: PAC learning of Boolean functions
- **Cryptography**: Goldreich-Levin Algorithm [GL89]
- **Circuit Complexity** [Håstad88, LMN93]
- **Decision Tree Complexity**: Huang's proof of the Sensitivity Conjecture [Hua19]
- **Quantum Complexity** [BBC+01]

---

## Learning Objectives

1. Understand what a Boolean function is and the {±1} convention
2. Learn the Fourier expansion as a multilinear polynomial
3. Understand orthogonality of characters (parity functions)
4. Apply Parseval's and Plancherel's identities
5. Use the inversion formula to compute Fourier coefficients

In [ ]:
# Run this cell first if using Google Colab
# !pip install boofun matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import boofun as bf
from boofun.analysis.fourier import (
    parseval_verify, 
    plancherel_inner_product,
    fourier_degree,
    spectral_norm
)
from boofun.visualization import BooleanFunctionVisualizer, plot_hypercube

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print(f"BooFun version: {bf.__version__}")

## 1.1 What is a Boolean Function?

A **Boolean function** is a function $f: \{0,1\}^n \to \{0,1\}$.

Boolean functions can model:
- **Pseudorandomness**: A test we want to fool
- **Combinatorics**: Set systems via indicator functions  
- **Social choice**: $n$ votes aggregated into a decision
- **Coding theory**: Error-correcting codes
- **Graph properties**: Properties of graphs encoded in binary

### The {±1} Convention

We typically use $\{+1, -1\}$ instead of $\{0, 1\}$:
- **+1** represents **False** (or 0)
- **-1** represents **True** (or 1)

The isomorphism: $b \mapsto (-1)^b$

This makes the Fourier expansion cleaner (characters become products of variables).

The domain $\{±1\}^n$ is called the **Boolean hypercube**.

In [ ]:
# Visualize the Boolean hypercube (Figure 1.1 from lecture)
maj_3 = bf.majority(3)
print("The 3-dimensional Boolean hypercube with MAJORITY₃:")
print("  Red vertices: f(x) = 1 (True)")
print("  Blue vertices: f(x) = 0 (False)")
plot_hypercube(maj_3, figsize=(8, 6))

In [ ]:
# Creating Boolean functions with boofun

# Example from lecture: max₂(x₁, x₂) = AND(x₁, x₂)
max_2 = bf.AND(2)
print("max₂(x₁, x₂) = AND(x₁, x₂) - outputs maximum of two Booleans")
print("  In {±1}: +1=False, -1=True, so max(-1,-1)=-1, max(+1,+1)=+1")
print()
print("Truth table (in {0,1} notation):")
print("  x₁  x₂  max₂")
for i in range(4):
    x1, x2 = (i >> 0) & 1, (i >> 1) & 1
    print(f"   {x1}   {x2}    {max_2.evaluate(i)}")

# Built-in functions
and_3 = bf.AND(3)
or_3 = bf.OR(3)
maj_3 = bf.majority(3)
parity_3 = bf.parity(3)

print("\nBuilt-in Boolean functions (n=3):")
print(f"  AND(1,1,1) = {and_3.evaluate(7)}")
print(f"  OR(0,0,1) = {or_3.evaluate(1)}")
print(f"  MAJORITY(1,1,0) = {maj_3.evaluate(6)}")
print(f"  PARITY(1,0,1) = {parity_3.evaluate(5)}")

## 1.2 The Fourier Expansion

**Theorem 1.3 (Fundamental Theorem of Boolean Functions):**

Every Boolean function $f: \{\pm 1\}^n \to \mathbb{R}$ can be uniquely represented as a **multilinear polynomial**:

$$f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \cdot \chi_S(x)$$

where:
- $[n] = \{1, \ldots, n\}$
- $\chi_S(x) = \prod_{i \in S} x_i$ is a **character** (parity/Walsh function)
- $\hat{f}(S)$ is the **Fourier coefficient** of $f$ on $S$

### Example 1.1: max₂ (AND function)

$$\text{max}_2(x_1, x_2) = \frac{1}{2} + \frac{1}{2}x_1 + \frac{1}{2}x_2 - \frac{1}{2}x_1 x_2$$

### Example 1.2: MAJORITY₃

$$\text{maj}_3(x_1, x_2, x_3) = \frac{1}{2}x_1 + \frac{1}{2}x_2 + \frac{1}{2}x_3 - \frac{1}{2}x_1 x_2 x_3$$

In [ ]:
# Computing Fourier coefficients is simple: just call f.fourier()

# Example 1.1: max₂ = AND₂
max_2 = bf.AND(2)
fourier_max2 = max_2.fourier()

print("Example 1.1: max₂ (AND function)")
print(f"  Fourier coefficients: {fourier_max2}")
print(f"    f̂(∅)   = {fourier_max2[0]:+.2f}  (index 0 = empty set)")
print(f"    f̂({{1}}) = {fourier_max2[1]:+.2f}  (index 1 = {{1}})")
print(f"    f̂({{2}}) = {fourier_max2[2]:+.2f}  (index 2 = {{2}})")
print(f"    f̂({{1,2}}) = {fourier_max2[3]:+.2f}  (index 3 = {{1,2}})")
print("  → max₂(x) = 1/2 + (1/2)x₁ + (1/2)x₂ - (1/2)x₁x₂  ✓")

print()

# Example 1.2: MAJORITY₃  
maj_3 = bf.majority(3)
fourier_maj3 = maj_3.fourier()

print("Example 1.2: MAJORITY₃")
print(f"  Fourier coefficients: {fourier_maj3}")
print("  Non-zero coefficients:")
print(f"    f̂({{1}}) = f̂({{2}}) = f̂({{3}}) = {fourier_maj3[1]:+.2f}")
print(f"    f̂({{1,2,3}}) = {fourier_maj3[7]:+.2f}")
print("  → maj₃(x) = (1/2)x₁ + (1/2)x₂ + (1/2)x₃ - (1/2)x₁x₂x₃  ✓")

## 1.3 Parseval's Identity

**Corollary 1.7 (Parseval's Identity):** For any $f: \{\pm 1\}^n \to \{\pm 1\}$:

$$\sum_{S \subseteq [n]} \hat{f}(S)^2 = 1$$

This is because $\langle f, f \rangle = \mathbb{E}[f(x)^2] = \mathbb{E}[1] = 1$ when $f$ takes values in $\{\pm 1\}$.

In [ ]:
# Verify Parseval's identity for several functions
functions = {
    "AND₃": bf.AND(3),
    "OR₃": bf.OR(3),
    "MAJORITY₃": bf.majority(3),
    "PARITY₃": bf.parity(3),
}

print("Parseval's Identity: Σ f̂(S)² = 1 for Boolean functions\n")
print(f"{'Function':<12} | {'Σ f̂(S)²':<10} | Parseval")
print("-" * 40)

for name, f in functions.items():
    # Direct API: f.fourier()
    fourier = f.fourier()
    sum_squares = sum(c**2 for c in fourier)
    verified = parseval_verify(f)
    status = "✓" if verified else "✗"
    print(f"{name:<12} | {sum_squares:<10.6f} | {status}")

## 1.4 Characters and the Inversion Formula

The **characters** $\chi_S(x) = \prod_{i \in S} x_i$ compute parity functions:
- $\chi_\emptyset(x) = 1$ (constant function)
- $\chi_{\{1\}}(x) = x_1$ (dictator on variable 1)
- $\chi_{\{1,2\}}(x) = x_1 x_2$ (parity of variables 1 and 2)

**Lemma 1.5**: The characters form an **orthonormal basis**:
$$\langle \chi_S, \chi_T \rangle = \begin{cases} 1 & \text{if } S = T \\ 0 & \text{otherwise} \end{cases}$$

**Inversion Formula** (how to compute Fourier coefficients):
$$\hat{f}(S) = \langle f, \chi_S \rangle = \mathbb{E}_{x}[f(x) \cdot \chi_S(x)]$$

In [ ]:
# Demonstrating the inversion formula: f̂(S) = E[f(x)·χ_S(x)]

def chi(S, x, n):
    """Compute χ_S(x) = product of x_i for i in S."""
    result = 1
    for i in range(n):
        if (S >> i) & 1:  # i is in S
            xi = 1 - 2 * ((x >> i) & 1)  # Convert to ±1
            result *= xi
    return result

def compute_fourier_coefficient(f, S, n):
    """Compute f̂(S) using the inversion formula."""
    total = 0
    for x in range(2**n):
        fx = 1 - 2 * f.evaluate(x)  # Convert to ±1
        total += fx * chi(S, x, n)
    return total / (2**n)

# Verify for MAJORITY₃
maj_3 = bf.majority(3)
n = 3

print("Inversion formula verification for MAJORITY₃:")
print("f̂(S) = E[f(x)·χ_S(x)]\n")

# Get actual coefficients from boofun
actual = maj_3.fourier()

for S in range(8):
    computed = compute_fourier_coefficient(maj_3, S, n)
    bits = [i+1 for i in range(n) if (S >> i) & 1]
    set_str = "{" + ",".join(map(str, bits)) + "}" if bits else "∅"
    
    if abs(computed) > 1e-10:
        print(f"  f̂({set_str:8}) = {computed:+.4f}  (computed)")
        print(f"           = {actual[S]:+.4f}  (from boofun) ✓")

## 1.5 Plancherel's Identity

**Corollary 1.6 (Plancherel's Identity):** For any $f, g: \{\pm 1\}^n \to \mathbb{R}$:

$$\langle f, g \rangle = \sum_{S \subseteq [n]} \hat{f}(S) \hat{g}(S)$$

The inner product in the "time domain" equals the inner product in the "frequency domain".

In [ ]:
# Verify Plancherel's identity
f = bf.AND(3)
g = bf.OR(3)

# Compute inner product directly: ⟨f, g⟩ = E[f(x)g(x)]
n = 3
direct_ip = 0
for x in range(1 << n):
    fx = 1 - 2 * f.evaluate(x)  # ±1
    gx = 1 - 2 * g.evaluate(x)  # ±1
    direct_ip += fx * gx
direct_ip /= (1 << n)

# Compute via Fourier domain
fourier_ip = plancherel_inner_product(f, g)

print("Plancherel's Identity verification:")
print(f"  Direct inner product ⟨AND, OR⟩ = {direct_ip:.6f}")
print(f"  Fourier inner product Σ f̂(S)ĝ(S) = {fourier_ip:.6f}")
print(f"  Match: {'✓' if abs(direct_ip - fourier_ip) < 1e-10 else '✗'}")

## 1.6 Influences and Spectral Properties

The Fourier expansion reveals structural properties:

- **Influence of variable $i$**: $\text{Inf}_i(f) = \sum_{S \ni i} \hat{f}(S)^2$
- **Total influence**: $\text{Inf}(f) = \sum_i \text{Inf}_i(f) = \sum_S |S| \cdot \hat{f}(S)^2$

In [ ]:
# Analyze spectral properties of MAJORITY₅
f = bf.majority(5)

print("Spectral Analysis of MAJORITY₅:")
print("=" * 40)

# Direct API: f.influences() and f.total_influence()
influences = f.influences()
total_inf = f.total_influence()

print(f"\nVariable influences:")
for i, inf in enumerate(influences):
    print(f"  Inf_{i+1}(f) = {inf:.4f}")

print(f"\nTotal influence: {total_inf:.4f}")

# Degree
deg = fourier_degree(f)
print(f"Fourier degree: {deg}")

## 1.7 Visualization: Fourier Spectrum

The `boofun` library includes rich visualization tools for exploring Boolean functions.
Let's visualize the Fourier spectrum and influences.

In [ ]:
# Visualizing influences with boofun

# Create a majority function and visualize its properties
maj5 = bf.majority(5)
viz = BooleanFunctionVisualizer(maj5)

# Plot influence of each variable
print("Variable influences for MAJORITY₅:")
fig = viz.plot_influences(figsize=(10, 4), show=False)
plt.title("Variable Influences for MAJORITY₅")
plt.tight_layout()
plt.show()

print("\nVisualization shows:")
print("  - Majority has equal influence for all variables (symmetric)")
print("  - This follows from the symmetry of the function")

In [ ]:
# Fourier spectrum visualization
viz_maj = BooleanFunctionVisualizer(maj5)

# Plot Fourier spectrum with spectral concentration
fig = viz_maj.plot_fourier_spectrum(figsize=(12, 5), show=True)

print("\nFourier spectrum analysis:")
print("  - Left: Distribution of |coefficients| by degree")
print("  - Right: Cumulative spectral weight (90%/99% thresholds)")
print("  - Majority has weight concentrated on degree 1 and 3 terms")

In [ ]:
# Comprehensive analysis dashboard
tribes = bf.tribes(2, 4)  # Tribes function
viz_tribes = BooleanFunctionVisualizer(tribes)

# Create a full dashboard showing multiple properties
fig = viz_tribes.create_dashboard(show=True)

print("\nDashboard includes:")
print("  - Variable influences (which inputs matter most)")
print("  - Fourier spectrum (frequency content)")
print("  - Summary statistics")
print("  - Noise stability curve (robustness to noise)")

## Summary

### Key Takeaways from Lecture 1:

1. **Boolean functions** $f: \{\pm 1\}^n \to \{\pm 1\}$ model many computational problems

2. **Fourier expansion**: Every Boolean function is a unique multilinear polynomial
   $$f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \chi_S(x)$$

3. **Characters** $\chi_S(x) = \prod_{i \in S} x_i$ form an orthonormal basis

4. **Inversion formula**: $\hat{f}(S) = \langle f, \chi_S \rangle = \mathbb{E}[f(x) \chi_S(x)]$

5. **Plancherel**: $\langle f, g \rangle = \sum_S \hat{f}(S) \hat{g}(S)$

6. **Parseval**: $\sum_S \hat{f}(S)^2 = 1$ for Boolean-valued functions

### Next: Property Testing (Lecture 2)
- The BLR linearity test
- If $f(x+y) \approx f(x) + f(y)$ for most inputs, is $f$ close to linear?

---

*This notebook was created for CS294-92 using the `boofun` library.*